# 최종 검사까지 데이터 병합

In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

op_20 = pd.read_csv("./data/ABH csv/op20.csv")
op_30 = pd.read_csv("./data/ABH csv/op30.csv")
op_40S = pd.read_csv("./data/ABH csv/op40S.csv")

op_20.columns = op_20.columns + "_op_20"
op_30.columns = op_30.columns + "_op_30"

In [2]:
# op_20, op_30 병합
op_20_30_merged = pd.merge(
    left=op_30, right=op_20, left_on="바코드1_op_30", right_on="바코드1_op_20"
)

merged_40_30_20 = pd.merge(
    left=op_40S, right=op_20_30_merged, left_on="바코드", right_on="바코드1_op_30"
)

In [3]:
# 측정 오류로 재측정한 경우가 있습니다. 마지막 검사만을 남깁니다.
common_ids = (
    merged_40_30_20[merged_40_30_20.duplicated(subset="바코드", keep=False)]
    .groupby("바코드")
    .apply(lambda x: x.index.tolist())
)

indices_to_keep = [indices[-1] for indices in common_ids]

final_indices = set(indices_to_keep) | set(
    merged_40_30_20[~merged_40_30_20["바코드"].duplicated(keep=False)].index
)
merged_40_30_20_deduped = merged_40_30_20.loc[list(final_indices)]

C:\Users\MoonDongWon\AppData\Local\Temp\ipykernel_4968\3919855630.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  common_ids = merged_40_30_20[merged_40_30_20.duplicated(subset='바코드', keep=False)].groupby('바코드').apply(lambda x: x.index.tolist())


In [4]:
rem_columns = ["차종", "차종번호", "바코드", "판정", "판정_op_20", "판정_op_30"]
hv_ul_columns = [
    "전장",
    "거리",
    "캡간",
    "좌1차피크",
    "우1차피크",
    "좌완료거리",
    "우완료거리",
]

op_20_to_40 = merged_40_30_20_deduped.loc[
    :,
    rem_columns
    + [
        col + ul + op
        for op in ("_op_20", "_op_30")
        for col in hv_ul_columns
        for ul in ("상한", "", "하한")
    ],
]

df = op_20_to_40.loc[:, op_20_to_40.columns.difference(["차종", "바코드"])].copy()

df["판정"] = df["판정"].apply(lambda x: 1 if x == "합격" else 0)
df["판정_op_20"] = df["판정_op_20"].apply(lambda x: 1 if x == "합격" else 0)
df["판정_op_30"] = df["판정_op_30"].apply(lambda x: 1 if x == "합격" else 0)

In [ ]:
df.to_csv("./data/op_20_to_40S.csv", index=False)